In [ ]:
# Setup input parameters
from datetime import datetime as dt
dbutils.widgets.text('bg_loadtimestamp', '')
bg_loadtimestamp = dbutils.widgets.get('bg_loadtimestamp')
bg_loadtimestamp_str = bg_loadtimestamp
if not bg_loadtimestamp:
    bg_loadtimestamp = 'CAST(NULL AS Timestamp)'
else:
    bg_loadtimestamp = f"CAST('{bg_loadtimestamp}' AS Timestamp)"




In [ ]:
# Setup logging
logger = spark._jvm.org.apache.log4j.Logger.getLogger('com.bigenius-x.application')
def info(targetName, message):
    logger.info(f'{targetName}: {message}')
    print(f"{dt.now().strftime('%Y/%m/%d, %H:%M:%S')} - {targetName}: {message}")




In [ ]:
# EntityCleanseLoader: Ent_orders_Entity Cleanse Loader_1

try:

    operation_metrics_collection = {}
    result_df = spark.sql(f"""
    INSERT
    OVERWRITE `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_orders` (
         `bg_sourcesystem`
        ,`bg_loadtimestamp`
        ,`bg_errorcount`
        ,`bg_effectivetimestamp`
        ,`order_id`
        ,`order_status`
        ,`order_total`
    )
    SELECT
         `bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,{bg_loadtimestamp} AS `bg_loadtimestamp`
        ,0 AS `bg_errorcount`
        ,`bg_source`.`bg_effectivetimestamp` AS `bg_effectivetimestamp`
        ,`bg_source`.`order_id` AS `order_id`
        ,`bg_source`.`order_status` AS `order_status`
        ,`bg_source`.`order_total` AS `order_total`
    FROM `{marc_databricks_initiative#cleanse#database_name}`.`{marc_databricks_initiative#cleanse#schema_name}`.`cls_en_ent_orders_source` AS `bg_source`
    """)
    RowCountInserted = result_df.select("num_inserted_rows").collect()[0][0]
    operation_metrics = result_df.toJSON().collect()
    operation_metrics_collection['reloadtarget_{marc_databricks_initiative#cleanse#database_name}_{marc_databricks_initiative#cleanse#schema_name}_cls_en_ent_orders'] = operation_metrics

except Exception as e:
    info('CLS_EN_Ent_orders_Loader', e)
    raise



In [ ]:
dbutils.notebook.exit(operation_metrics_collection)
